In [1]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
traffic_data = pd.read_csv('../Traffic data/SCOOT/data_470_hourly/flows/GD030A_S.csv')

## 1. Recover timestamp

In [3]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [4]:
# Apply the recover_timestamp function to recover the full time series
traffic_full = recover_timestamp(traffic_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


## 2. Train, validate, test data split

In [21]:
# train_set = traffic_full[:'2022-02-28 23:00:00']
# valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
# test_set = traffic_full['2023-01-01 00:00:00':]
train_set = traffic_full['2022-06-03 00:00:00':'2023-03-31 23:00:00']
valid_set = traffic_full['2023-04-01 00:00:00':'2023-06-30 23:00:00']
test_set = traffic_full['2023-07-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full['2022-06-03 00:00:00':])))

Proportion of train_set : 0.6227
Proportion of valid_set : 0.1876
Proportion of test_set : 0.1897


In [22]:
train_set

,date,time,flow
2022-06-03 00:00:00,2022-06-03,0.0,102.0
2022-06-03 01:00:00,2022-06-03,1.0,127.0
2022-06-03 02:00:00,2022-06-03,2.0,60.0
2022-06-03 03:00:00,2022-06-03,3.0,12.0
2022-06-03 04:00:00,2022-06-03,4.0,9.0
...,...,...,...
2023-03-31 19:00:00,2023-03-31,19.0,119.0
2023-03-31 20:00:00,2023-03-31,20.0,113.0
2023-03-31 21:00:00,2023-03-31,21.0,104.0
2023-03-31 22:00:00,2023-03-31,22.0,83.0


## 3. Normalise the data 

In [23]:
# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data's 'flow' feature
scaler.fit(train_set[['flow']])

# Transform the 'flow' feature in all datasets
train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
test_set.loc[:, 'flow_scaled'] = scaler.transform(test_set[['flow']])

C:\Users\Amanda\AppData\Local\Temp\ipykernel_6540\1100395100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
C:\Users\Amanda\AppData\Local\Temp\ipykernel_6540\1100395100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
C:\Users\Amanda\AppData\Local\Temp\ipykernel_6540\1100395100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

## 4. Split the data into X and y



In [10]:
# Define the create_multi_step_sequence function
def create_multi_step_sequence(data, last_n_steps, day_lag, week_lag, n_future_steps):
    """
    Create input sequences from data using multiple time windows and multiple future steps as output.

    Parameters:
    - data: The time series data (1D array or list with possible NaN values)
    - last_n_steps: Number of most recent steps to use as part of the input (default 12)
    - day_lag: Time lag for the last day's value (24 hours ago, default 24)
    - week_lag: Time lag for the last week's value (168 hours ago, default 168)
    - n_future_steps: Number of future steps to predict (default 6)

    Returns:
    - X: Input features combining last_n_steps, last day's value, and last week's value
    - y: Output labels (shape: [samples, n_future_steps])
    """
    X, y = [], []

    # Loop over the data to create the input-output pairs
    for i in range(max(last_n_steps, day_lag, week_lag), len(data) - n_future_steps):
        # Input sequence of the last `last_n_steps` observations
        input_seq = data['flow'].values[i - last_n_steps:i]

        # Last day's observation (24 hours ago)
        last_day_value = data['flow'].values[i - day_lag]

        # Last week's observation (168 hours ago)
        last_week_value = data['flow'].values[i - week_lag]

        # Output (next `n_future_steps` observations)
        output_seq = data['flow'].values[i:i + n_future_steps]

        # Check if any NaN values exist in the input or output sequences
        if not np.isnan(input_seq).any() and not np.isnan(last_day_value) and not np.isnan(last_week_value) and not np.isnan(output_seq).any():
            # Combine the features: last_n_steps, last_day_value, last_week_value
            X.append(np.concatenate([input_seq, [last_day_value], [last_week_value]]))
            y.append(output_seq)

    # Convert to numpy arrays and reshape X to match CNN expected input (samples, timesteps, features)
    X = np.array(X).reshape(-1, last_n_steps + 2, 1)  # Add the 2 additional features (last_day_value, last_week_value)
    y = np.array(y).reshape(-1, n_future_steps)  # Multiple output steps

    # Convert the entire input and output into a pandas DataFrame with appropriate column names for multi-step prediction

    # Define column names for the input DataFrame
    input_columns = [f'Step_{i}_back' for i in range(last_n_steps, 0, -1)] + ['Day_1_back', 'Week_1_back']

    # Create a DataFrame for the input (all rows)
    df_X = pd.DataFrame(X.reshape(X.shape[0], 14), columns=input_columns)

    # Create a DataFrame for the output (all rows), where each column represents one of the next 6 steps
    output_columns = [f'Next_Step_{i}' for i in range(0, n_future_steps)]
    df_y = pd.DataFrame(y, columns=output_columns)

    return X, y, df_X, df_y

# Code to predict next 6 steps simultaneously

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step) and 5 steps ahead

In [11]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)

ValueError: cannot reshape array of size 1141040 into shape (168,1)

## 5. Define LSTM model

In [12]:
def create_lstm_model_multi_step(input_shape, n_outputs, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_outputs))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',  # Mean Squared Error loss for regression
        optimizer=optimizer,
        metrics=['mae']  # Mean Absolute Error as a metric
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]

## 6. Set Up Hyperparameter Grid

In [13]:
# Hyperparameter options
units_list = [50, 100, 200]
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

# Create all possible combinations
hyperparameter_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

## 7. Train the Model with Hyperparameter Tuning

In [14]:
# Initialize variables to store the best model and hyperparameters
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (units, dropout_rate, learning_rate, batch_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    # Create the LSTM model with the current hyperparameters
    model = create_lstm_model_multi_step(
        input_shape=input_shape,
        n_outputs=n_outputs,
        units=units,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate
    )

    # Initialize EarlyStopping
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'units': units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")


Combination 1/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=32


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation MAE: 0.0602

Combination 2/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=64
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0600

Combination 3/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=128
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0600

Combination 4/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0618

Combination 5/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=64
Restoring model weights from the end of the best epoch: 44.
Validation MAE: 0.0634

Combination 6/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=128
Restoring model weights from the end of the b

Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0708

Combination 46/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=32
Epoch 27: early stopping
Restoring model weights from the end of the best epoch: 17.
Validation MAE: 0.0605

Combination 47/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=64
Epoch 31: early stopping
Restoring model weights from the end of the best epoch: 21.
Validation MAE: 0.0606

Combination 48/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=128
Epoch 37: early stopping
Restoring model weights from the end of the best epoch: 27.
Validation MAE: 0.0608

Combination 49/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0616

Combination 50/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=64
Restoring model weights fro

In [ ]:
13:33

Best Hyperparameters for full time:
units: 200
dropout_rate: 0.5
learning_rate: 0.01
batch_size: 32
Best Validation MAE: 0.0503

Best Hyperparameters for after covid:
units: 100
dropout_rate: 0.3
learning_rate: 0.01
batch_size: 128
Best Validation MAE: 0.0595

## 8. Make predictions

In [15]:
# Make predictions
y_pred_scaled = best_model.predict(x_test_scaled)

# Reshape for inverse scaling
y_pred_reshaped = y_pred_scaled.reshape(-1, 1)
y_test_reshaped = y_test_scaled.reshape(-1, 1)

# Inverse transform
y_pred_inverse = y_scaler.inverse_transform(y_pred_reshaped).reshape(n_test_samples, n_outputs)
y_test_inverse = y_scaler.inverse_transform(y_test_reshaped).reshape(n_test_samples, n_outputs)

y_test_flat = y_test_inverse.flatten()
y_pred_flat = y_pred_inverse.flatten()

66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


## 9. Evaluating the LSTM Model

In [16]:
# Compute Metrics for Each Time Step

for i in range(n_outputs):
    y_true = y_test_inverse[:, i]
    y_pred = y_pred_inverse[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Mean Absolute Percentage Error (MAPE)
    # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
    epsilon = 1e-10
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")


Time Step 1 Evaluation Metrics:
RMSE: 28.4454
MAE: 20.8450
MAPE: 19.44%

Time Step 2 Evaluation Metrics:
RMSE: 36.2563
MAE: 26.7673
MAPE: 25.02%

Time Step 3 Evaluation Metrics:
RMSE: 43.7203
MAE: 33.1341
MAPE: 29.97%

Time Step 4 Evaluation Metrics:
RMSE: 49.3573
MAE: 38.0972
MAPE: 35.19%

Time Step 5 Evaluation Metrics:
RMSE: 53.2812
MAE: 41.5888
MAPE: 39.42%

Time Step 6 Evaluation Metrics:
RMSE: 53.3926
MAE: 42.0135
MAPE: 44.19%


# Code to predict next 6 steps step-by-step

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step)

## 4. Create input and output data

In [29]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)

In [42]:
X_train.shape, X_valid.shape, X_test.shape

((19570, 14, 1), (6907, 14, 1), (5623, 14, 1))

## 5. Normalise the data after split (step-by-step)

Normalise X

In [31]:
# Separate scalers for inputs and outputs
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape x_train to 2D for scaling
n_samples, n_timesteps, n_features = X_train.shape
x_train_reshaped = X_train.reshape(-1, n_features)  # Shape: (n_samples * n_timesteps, n_features)
# Fit the scaler on the training data
x_scaler.fit(x_train_reshaped)
# Transform the training data
x_train_scaled = x_scaler.transform(x_train_reshaped)
# Reshape back to original shape
x_train_scaled = x_train_scaled.reshape(n_samples, n_timesteps, n_features)

# x_val
n_val_samples = X_valid.shape[0]
x_val_reshaped = X_valid.reshape(-1, n_features)
x_val_scaled = x_scaler.transform(x_val_reshaped)
x_val_scaled = x_val_scaled.reshape(n_val_samples, n_timesteps, n_features)

# x_test
n_test_samples = X_test.shape[0]
x_test_reshaped = X_test.reshape(-1, n_features)
x_test_scaled = x_scaler.transform(x_test_reshaped)
x_test_scaled = x_test_scaled.reshape(n_test_samples, n_timesteps, n_features)

Normalise y

In [32]:
# Reshape y_train to 2D for scaling
y_train_reshaped = y_train.reshape(-1, 1)  # Shape: (n_samples * n_outputs, 1)
# Fit the scaler on the training data
y_scaler.fit(y_train_reshaped)
# Transform the training data
y_train_scaled = y_scaler.transform(y_train_reshaped)
# Reshape back to original shape
y_train_scaled = y_train_scaled.reshape(n_samples, y_train.shape[1])

# y_val
y_val_reshaped = y_valid.reshape(-1, 1)
y_val_scaled = y_scaler.transform(y_val_reshaped)
y_val_scaled = y_val_scaled.reshape(n_val_samples, y_valid.shape[1])

# y_test
y_test_reshaped = y_test.reshape(-1, 1)
y_test_scaled = y_scaler.transform(y_test_reshaped)
y_test_scaled = y_test_scaled.reshape(n_test_samples, y_test.shape[1])

## 6. Build LSTM

In [21]:
def create_lstm_model(input_shape, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1)) # Output layer for one-step prediction
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])

## 7. Hyperparameter Tuning

In [22]:
# Hyperparameter options
units_list = [50, 100, 200]
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

# Create all possible combinations
hyperparameter_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

In [23]:
# Initialize variables to store the best model and hyperparameters
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (units, dropout_rate, learning_rate, batch_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    # Create the LSTM model with the current hyperparameters
    model = create_lstm_model(
        input_shape=input_shape,
        units=units,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate
    )

    # Initialize EarlyStopping
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'units': units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")



Combination 1/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=32


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 40: early stopping
Restoring model weights from the end of the best epoch: 30.
Validation MAE: 0.0452

Combination 2/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=64
Epoch 44: early stopping
Restoring model weights from the end of the best epoch: 34.
Validation MAE: 0.0452

Combination 3/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=128
Epoch 23: early stopping
Restoring model weights from the end of the best epoch: 13.
Validation MAE: 0.0466

Combination 4/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 48.
Validation MAE: 0.0450

Combination 5/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=64
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0463

Combination 6/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=128
Restoring model weights from the end of the b

Restoring model weights from the end of the best epoch: 46.
Validation MAE: 0.0513

Combination 46/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=32
Epoch 29: early stopping
Restoring model weights from the end of the best epoch: 19.
Validation MAE: 0.0466

Combination 47/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=64
Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 15.
Validation MAE: 0.0460

Combination 48/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=128
Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation MAE: 0.0458

Combination 49/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0463

Combination 50/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=64
Epoch 34: early stopping
Re

Best Hyperparameters full time:
units: 50
dropout_rate: 0
learning_rate: 0.01
batch_size: 128
Best Validation MAE: 0.0388

Best Hyperparameters after covid:
units: 100
dropout_rate: 0
learning_rate: 0.001
batch_size: 32
Best Validation MAE: 0.0450

#### Recreate the model¶

In [33]:
def best_lstm_model(input_shape, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1)) # Output layer for one-step prediction
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

In [34]:
input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
units=50
dropout_rate=0
learning_rate=0.01
batch_size=128

#### Retrain the model

In [35]:
# Initialize EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mae',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [37]:
# Train the model
history = best_model.fit(
    x_train_scaled, y_train_scaled,
    epochs=50,  # You can adjust this as needed
    batch_size=batch_size,
    validation_data=(x_val_scaled, y_val_scaled),
    callbacks=[early_stopping],
    verbose=1  # Set to 1 to see detailed training output
)

Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0044 - mae: 0.0474 - val_loss: 0.0039 - val_mae: 0.0405
Epoch 2/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0043 - mae: 0.0465 - val_loss: 0.0039 - val_mae: 0.0406
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0467 - val_loss: 0.0040 - val_mae: 0.0432
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0044 - mae: 0.0472 - val_loss: 0.0039 - val_mae: 0.0401
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0469 - val_loss: 0.0040 - val_mae: 0.0430
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0044 - mae: 0.0473 - val_loss: 0.0039 - val_mae: 0.0399
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0464 - val_loss: 0.0039 - val_mae: 0.0406
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0045 - mae: 0.0475 - val_loss: 0.0039 - val_mae: 0.0416
Epoch 9/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/

## 8. Recursive Forecasting with LSTM (step-by-step)

In [38]:
def recursive_forecast(model, x_test_scaled, start_index, n_steps, x_scaler, y_scaler):
    """
    Perform recursive forecasting using the trained model.

    Parameters:
    - model: Trained model
    - x_test_scaled: The scaled test input data (shape: n_samples, n_timesteps, n_features)
    - start_index: The starting index in the test data
    - n_steps: Number of future steps to predict
    - x_scaler: Scaler used for input features
    - y_scaler: Scaler used for target variable

    Returns:
    - predictions: List of predicted values (in original scale)
    """
    predictions = []
    scaled_predictions = []
    current_input = x_test_scaled[start_index].copy()  # Shape: (n_timesteps, n_features)

    for step in range(n_steps):
        # Reshape to (1, n_timesteps, n_features) for prediction
        input_seq = current_input.reshape((1, current_input.shape[0], current_input.shape[1]))

        # Predict the next time step (scaled)
        yhat_scaled = model.predict(input_seq, verbose=0)  # Shape: (1, 1)

        # Inverse transform the prediction to original scale
        yhat = y_scaler.inverse_transform(yhat_scaled)[0, 0]
        
        # Transform the prediction back to input feature scale for lag features
        yhat_for_input = x_scaler.transform(yhat.reshape(-1, 1))[0, 0]

        # Append predictions
        predictions.append(yhat)
        scaled_predictions.append(yhat_for_input)

        # Move to the next time step in x_test_scaled
        next_index = start_index + step + 1
        if next_index < len(x_test_scaled):
            # Use features from the next time step
            next_input = x_test_scaled[next_index].copy()
        else:
            # Reached the end of x_test_scaled
            break

        # Update lag features with available scaled predictions
        for lag in range(1, min(step + 1, 6) + 1):
            feature_index = 12 - lag  # lag1 is at index 11
            next_input[feature_index, 0] = scaled_predictions[-lag]

        # Keep lag24 and lag168 as they are, or update if necessary

        # Set current_input for next iteration
        current_input = next_input

    return predictions

## 9. Make step-by-step prediction

In [39]:
# Number of steps to predict
n_steps = 6

# Initialize lists to store predictions and actual values
all_predictions = []
all_actuals = []

# Ensure we have enough data for recursive predictions
n_test_samples = x_test_scaled.shape[0]

for i in range(n_test_samples - n_steps):
    # Perform recursive forecasting
    predictions = recursive_forecast(
        model=best_model,
        x_test_scaled=x_test_scaled,
        start_index=i,
        n_steps=n_steps,
        x_scaler=x_scaler,
        y_scaler=y_scaler
    )

    # Get the actual future values (in original scale)
    actual_values = y_test[i+1:i + len(predictions) + 1].flatten()

    # Store the predictions and actual values
    all_predictions.append(predictions)
    all_actuals.append(actual_values)

# Convert lists to numpy arrays
all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

## 10. Evaluating the LSTM Model

In [40]:
# Compute evaluation metrics
epsilon = 1e-10  # To avoid division by zero in MAPE

for i in range(n_steps):
    y_true = all_actuals[:, i]
    y_pred = all_predictions[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Mean Absolute Percentage Error (MAPE)
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")


Time Step 1 Evaluation Metrics:
RMSE: 41.4159
MAE: 29.8309
MAPE: 32.89%

Time Step 2 Evaluation Metrics:
RMSE: 44.7596
MAE: 31.9444
MAPE: 36.16%

Time Step 3 Evaluation Metrics:
RMSE: 47.2820
MAE: 33.4781
MAPE: 39.15%

Time Step 4 Evaluation Metrics:
RMSE: 49.5900
MAE: 34.8770
MAPE: 41.75%

Time Step 5 Evaluation Metrics:
RMSE: 51.5312
MAE: 35.9326
MAPE: 43.32%

Time Step 6 Evaluation Metrics:
RMSE: 53.2142
MAE: 36.6299
MAPE: 44.43%


In [ ]:
step-by-step full date